#0.環境構築

##0-1.まずは以下のサイトよりDeepInfraのアカウントを作成してください。

https://deepinfra.com/

##0-2.以下よりAPI KEYを取得してセルにセットしてください。

https://deepinfra.com/dash/api_keys

In [ ]:
DEEPINFRA_API_KEY = "pbXLVO0CmR6d66hRTpb6Lt8H4q14f3RF"

##0-3.Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


エクセルファイルの保存先パスを設定してください。

In [ ]:
G_FOLDER = "/content/drive/MyDrive"

##0-4.以下のコードで必要なライブラリのインストールと、APIのテスト実行を行います。

In [ ]:
!pip install -qU openai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
# Assume openai>=1.0.0
from openai import OpenAI

# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=DEEPINFRA_API_KEY,
    base_url="https://api.deepinfra.com/v1/openai",
)

In [ ]:
chat_completion = openai.chat.completions.create(
    model="mistralai/Mixtral-8x22B-Instruct-v0.1",
    messages=[{"role": "system", "content": "あなたは日本語で回答するAIボットです。"},
     {"role": "user", "content": "以下の文章が文法的に正しいかどうかを評価してください。文法的に正しい場合には1を、誤りを含んでいる場合には0を選択してください。  本件に関しましては、貴社のご要望を踏まえて最適なソリューションをご提案させていただきます次第でございます。"}],
)

print(chat_completion.choices[0].message.content)

 1

なお、この文章は日本語の敬語表現を含み、文法的には正しいと判断されますが、文脈や使い運びによっては適切ではない場合があることにはご注意ください。


#1.データ生成処理の実行

##1-1.プロンプト準備

言語理解でのプロンプト例

In [1]:
PROMPT_TEMPLATE = """
以下のタスク指示例を参考に、3種類の多様なinputとoutputを考えてください。これらのタスク指示はGPTモデルに与えられ、その指示を完了するためのGPTモデルを評価します。

要件は以下の通りです：
1. inputはビジネスの現場で用いられる文書とoutputはそれの要約を生成してください．
2. outputはビジネス現場で用いられるような適切な表現で生成してください．
3. inputは必ず1000文字以上の長文を生成し，outputにはinputの文章を要約した文章を端的に生成してください．
4. 同じ動詞や形容詞などを繰り返さないようにして、多様性を最大限に引き出してください。
5. 出力は指示に対する適切な応答であるべきです。出力はinputの回答となっていることを確認してください．
6. 生成するinputとoutputは全て必ず日本語で生成してください。
7. 回答フォーマット例に記載のJSON形式で回答してください。

タスク指示例：
$FewShotExamples

回答フォーマット例(JSON):
[
  {
    "id": 1,
    "instruction": "貴方はアシスタントです．与えられた文章を適切に要約してください．",
    "input": "<入力文章>",
    "output": "<回答文章>"
  },
  {
    "id": 2,
    "instruction": "貴方はアシスタントです．与えられた文章を適切に要約してください．",
    "input": "<入力文章>",
    "output": "<回答文章>"
  },
  {
    "id": 3,
    "instruction": "貴方はアシスタントです．与えられた文章を適切に要約してください．",
    "input": "<入力文章>",
    "output": "<回答文章>"
  }
]
"""

In [2]:
FEWSHOT_EXAMPLES = [
  {
    "id": 1,
    "instruction": "貴方はアシスタントです．与えられた文章を適切に要約してください．",
    "input": "我が国は、昨年の出生数が80万人を割り込むなど、急速に少子化が進展しており、今後、本格的な少子高齢化・人口減少時代を迎えようとしている。特に高齢期を中心に単身世帯者数は増加し、人との交流の意識も希薄化していく中で、孤独・孤立の問題が顕在化・深刻化している。また、社会保障をめぐるニーズや、人材など活用が可能な資源の状況は地域により大きく異なってくる。大都市を中心に人口の集中が見られ、人口減少が急速に進む地域では、地域における支え合いの機能が低下し、日常生活の維持も困難になってくる可能性も想定される。また、現役時代には職場とのつながりが高く地域とのつながりが低い傾向が見られる中で、高齢世代の未婚率の上昇が進んでおり、今後、高齢世代と地域とのつながりが一層弱くなることが懸念される。地域の人々が地域社会とつながりながら安心して生活を送ることができるように、地域ごとの特性に応じて取り組むべき課題を摘出し、解決の手法や仕組みを考察していくことが重要となるだろう。一方で、社会に対する貢献意識や人とのつながりに対する意識は比較的高い様子もうかがえるため、より多くの方が地域社会において何らかの役割を発揮できる環境整備も求められる。",
    "output": "我が国は少子高齢化・人口減少時代を迎えようとしています．特に大都市ではない地域では未婚率の上昇などが要因となり高齢世代と地域とのつながりが弱くなっていくことが懸念されています．そのため，課題の認識と解決法の考察，多くの人が地域社会に対して貢献できる環境構築が求められています．"
  },
  {
    "id": 2,
    "instruction": "貴方はアシスタントです．与えられた文章を適切に要約してください．",
    "input": """「経済安全保障」という概念は、冷戦直後に米国のクリントン政権が、米国の政策課題の中心として経済の重要性を強調するために用いたことから急速に一般化した56。E.H.Carr（1939）は、1930年代末に「政治と経済の分離」を「幻想」と批判し、「政治経済学」を唱導していた57が、冷戦時代には経済と安全保障の分離が当然視され、安全保障は専ら軍事的観点から考察されてきた。こうした中、冷戦が終結し、主要国間の国家中心、軍事中心の伝統的な安全保障問題が遠景に退く中で、国同士の相互依存の深まりから他国の経済政策や国際経済体制の運営・管理に一段と敏感になり、国外から国内経済への影響を「安全保障」問題と表現する度合いが高まってきたからと考えられる。さらに、2018年8月に成立した米国の国防権限法により、米国の輸出管理の強化および安全保障上の投資審査の強化が図られたことは、更に経済と安全保障の密接な結びつきを印象づけた。

「経済安全保障」という言葉は様々な意味で用いられているが58、「国家安全保障戦略」（2022年12月閣議決定）で「我が国の平和と安全や経済的な繁栄等の国益を経済上の措置を講じ確保すること」と定義された。本項では、まず、「経済安全保障」を考えるにあたっていくつかの論考を紹介する。

鈴木（2022）は、「経済安全保障」について、「供給の安全保障」「技術の不拡散の安全保障」「他国の規制からの安全保障」と三つに分類している59。「供給の安保障」はすなわちサプライチェーンの安全保障であり、2022年5月に成立した「経済安全保障推進法」の四つの柱の一つにもなっている。他国においてもサプライチェーン見直しの動きは顕著であり、例えば米国バイデン政権は2021年2月に重要製品の国内サプライチェーンを強化するための大統領令14017号に署名し、半導体製造、バッテリー、レアアース、医薬品等4分野の重要製品に関するサプライチェーンのリスク特定とリスクへの対処方法をまとめた報告書を同年6月にまとめさせた60。さらに、翌年2月にはエネルギー、運輸、農産物・食料生産、公衆衛生、情報通信技術、防衛の6分野についてもサプライチェーンを強化する戦略を含めた報告書がまとめられた61。

「技術の不拡散の安全保障」は貿易等を通じて技術が拡散・流出することで、他国の軍事能力が強化されたり、我が国の国際競争力が低下することが懸念される。米中対立は新興技術を巡る技術覇権競争の側面も大きく、我が国としてもこのような世界的な技術覇権競争の中で、「技術の不拡散の安全保障」の重要性が高まっている。

「他国の規制からの安全保障」については、例えば、米国の「グローバル・マグニツキー人権問責法」とそれに対抗する中国の「反外国制裁法」を巡る動向が挙げられる。このような他国の法律や規制はコントロールすることが難しく、日本企業はこれらの法律や規制にあわせて企業行動を変更させる必要を迫られる。

飯田（2023）は、経済安全保障を「経済が軍事力に影響する場合」及び「経済が交渉力に影響する場合」の二つの類型に整理している62。前者は、輸出管理、投資審査、武器移転、武器の国産化などが挙げられ、後者は経済制裁・経済的共生、エネルギー・食料安全保障、関与政策、サプライチェーンの多角化・強靱化などが挙げられる。""",
    "output": "「経済安全保障」はアメリカのクリントン政権が，アメリカ中心の経済の重要性を強調するために用いたため一般化した．「経済安全保障」にはいくつかの論考があり鈴木は「経済安全保障」は「供給の安全保障」，「技術の不拡散の安全保障」，「他国の規制からの安全保障」に分けている．一方で，飯田は「経済が軍事力に影響する場合」及び「経済が交渉力に影響する場合」の2つに分けている．"
  },
  {
    "id": 3,
    "instruction": "貴方はアシスタントです．与えられた文章を適切に要約してください．",
    "input": """ア　デジタル・トランスフォーメーションの取組状況
DXに関する取組を進めている企業の割合（「全社戦略に基づき、全社的にDXに取り組んでいる」、「全社戦略に基づき、一部の部門においてDXに取り組んでいる」、「部署ごとに個別でDXに取り組んでいる」の合計値）は、日本企業は約56％であるのに対し、米国企業は約79％と、日本企業の方が低かった（図表3-8-2-1）。DXに取り組む目的については、日本企業は「生産性向上」が約75％と最多であるのに対し、中国企業は「データ分析・活用」が約80％と最多である（図表3-8-2-2）。

イ　デジタル・トランスフォーメーションの効果
デジタル化の効果を「新規ビジネス創出」、「生産性向上」、「データ分析・活用」及び「商品・サービスの差別化」の観点に分けて調査した12ところ、各観点に共通して、日本企業は「期待以上」の回答が米国・中国・ドイツの3か国と比べて少なく、一方で「期待するほどの効果を得られていない」の回答は4か国の中で最も多い。

ウ　デジタル・トランスフォーメーションを推進する上での課題
デジタル化を進める上での課題・障壁として、日本企業は「人材不足（67.6％）」の回答が米国・中国・ドイツの3か国に比べて非常に多く、次いで「デジタル技術の知識・リテラシー不足（44.8％）」と、人材に関する課題・障壁が多い（図表3-8-2-3）。

また、各国の企業が保有するデジタル人材（「CIOやCDO等のデジタル化の主導者」、「AI・データ解析の専門家」）の不足状況については、日本企業は両デジタル人材に共通して「大いに不足している」と「多少不足している」を合計すると50％を超える結果となり、全体的にデジタル人材が不足している状況にある。特に「AI・データ解析の専門家」は「大いに不足している」が30％を超え、米国やドイツと比べると不足状況が深刻である。

各国の企業でデジタル人材が不足する理由については、日本企業は両デジタル人材に共通して「デジタル人材を採用する体制が整っていない」と「デジタル人材を育成する体制が整っていない」が約40％と多い。また、各国の企業のデジタル人材の確保に向けた取組について質的側面（必要なスキルの確保）から調査したところ、米国企業は「採用（新規・中途両方を含む）」が最も多い一方、日本企業は「社内の既存人材の配置転換や育成」が最も多い。""",
    "output": "「経済安全保障」はアメリカのクリントン政権が，アメリカ中心の経済の重要性を強調するために用いたため一般化した．「経済安全保障」にはいくつかの論考があり鈴木は「経済安全保障」は「供給の安全保障」，「技術の不拡散の安全保障」，「他国の規制からの安全保障」に分けている．一方で，飯田は「経済が軍事力に影響する場合」及び「経済が交渉力に影響する場合」の2つに分けている．"
  }
]

In [4]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(FewShotExamples=FEWSHOT_EXAMPLES)
print(formatted_prompt_template)


以下のタスク指示例を参考に、3種類の多様なinputとoutputを考えてください。これらのタスク指示はGPTモデルに与えられ、その指示を完了するためのGPTモデルを評価します。

要件は以下の通りです：
1. inputはビジネスの現場で用いられる文書とoutputはそれの要約を生成してください．
2. outputはビジネス現場で用いられるような適切な表現で生成してください．
3. inputは必ず1000文字以上の長文を生成し，outputにはinputの文章を要約した文章を端的に生成してください．
4. 同じ動詞や形容詞などを繰り返さないようにして、多様性を最大限に引き出してください。
5. 出力は指示に対する適切な応答であるべきです。出力はinputの回答となっていることを確認してください．
6. 生成するinputとoutputは全て必ず日本語で生成してください。
7. 回答フォーマット例に記載のJSON形式で回答してください。

タスク指示例：
[{'id': 1, 'instruction': '貴方はアシスタントです．与えられた文章を適切に要約してください．', 'input': '我が国は、昨年の出生数が80万人を割り込むなど、急速に少子化が進展しており、今後、本格的な少子高齢化・人口減少時代を迎えようとしている。特に高齢期を中心に単身世帯者数は増加し、人との交流の意識も希薄化していく中で、孤独・孤立の問題が顕在化・深刻化している。また、社会保障をめぐるニーズや、人材など活用が可能な資源の状況は地域により大きく異なってくる。大都市を中心に人口の集中が見られ、人口減少が急速に進む地域では、地域における支え合いの機能が低下し、日常生活の維持も困難になってくる可能性も想定される。また、現役時代には職場とのつながりが高く地域とのつながりが低い傾向が見られる中で、高齢世代の未婚率の上昇が進んでおり、今後、高齢世代と地域とのつながりが一層弱くなることが懸念される。地域の人々が地域社会とつながりながら安心して生活を送ることができるように、地域ごとの特性に応じて取り組むべき課題を摘出し、解決の手法や仕組みを考察していくことが重要となるだろう。一方で、社会に対する貢献意識や人とのつながりに対する意識は比較的高い様子もうかがえるため、より多くの方が地域社会において

##1-2.データ生成処理の実行

パラメーターをセット

In [ ]:
TEMPERATURE=1 #temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
MAX_TOKENS=10000
LOOP_NUM = 50 #1回あたり10件データを生成

In [ ]:
def run_model(PROMPT_TEMPLATE, TEMPERATURE=1, MAX_TOKENS=10000):

    chat_completion = openai.chat.completions.create(
        model="mistralai/Mixtral-8x22B-Instruct-v0.1",
        messages=[{"role": "system", "content": "あなたは日本語の学習データセットを生成する専門家です。"},
         {"role": "user", "content": PROMPT_TEMPLATE}],
        temperature=TEMPERATURE,#temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
        top_p=0.9,#0 < top_p ≤ 1 Sample from the set of tokens with highest probability such that sum of probabilies is higher than p. Lower values focus on the most probable tokens.Higher values sample more low-probability tokens
        max_tokens=MAX_TOKENS,
        frequency_penalty=1.3,#-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on how many times they appear in the text so far, increasing the model's likelihood to talk about new topics.
        presence_penalty=1.3 #-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.
        )

    output = chat_completion.choices[0].message.content

    return output

In [ ]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(FewShotExamples=FEWSHOT_EXAMPLES)

for i in range(LOOP_NUM):

    ### JSON形式で出力が出てこないケースがあるため、3回までリトライ ###
    for retry_i in range(3):

        ### LLMによるデータ生成処理 ###
        output = run_model(formatted_prompt_template, TEMPERATURE, MAX_TOKENS)
        print(output)

        try:
            #output = json.loads(output)
            df_output = pd.read_json(output)
            df_list.append(df_output)
            print(f"Success parsing JSON")
            break
        except Exception as e:
            print(e)
            print(f"Error parsing JSON: {retry_i}")
            continue


df_final = pd.concat(df_list)

df_final.head()

 [
  {
    "id": 1,
    "instruction": "貴方はアシスタントです．与えられた文章を適切に要約してください．",
    "input": "日本の情報通信技術（ICT）が国内経��PERTY及び就IT業界への意識的参入やその妨げに関する問題点を解明するため、企業、民間団体、高等学校の生徒・教員・管理者および社会につながる非営利活動団体を対象としたアンケート調査を実施しました。調査結果、IT業界への参入意識が少なく、情報技術の適性を持つ高等学校生徒が、自身の将来の業務にITを活用する可能性を意識せず、求められているICTスキルを習得していないことが分かりました。一方で、企業や民間団体では、IT人材不足を認識しており、ICTスキルの確保や取り組みが進められています。さらに、調査結果より分析し、日本の情報通信技術（ICT）の発展及び未来に向けた企業、民間団体、高等学校の取り組みとして、情報技術を自ら学び続ける文化の培養や教育機関と産業界との連携、卒業後も学び続ける環境を整えるなど、情報技術に対する意識を高め、スキル不足の解消を図りました。",
    "output": "調査結果より日本の情報通信技術（ICT）が国内経済及び就IT業界への意識的参入に関する問題点を解明しました。IT業界への参入意識が少なく、ICTスキルの習得が不足している高等学校生徒が見られました。その一方で、企業や民間団体ではIT人材不足を認識しており、ICTスキルの確保や取り組みが進められています。解決策として、情報技術を自ら学び続ける文化の培養や教育機関と産業界との連携、卒業後も学び続ける環境を整えるなど、情報技術に対する意識を高め、スキル不足の解消を図りました。"
  },
  {
    "id": 2,
    "instruction": "貴方はアシスタントです．与えられた文章を適切に要約してください．",
    "input": "日本の経済成長は明確な安定性を示しており、世界的なサプライチェーン危機やCOVID-19パンデミックによる影響にも耐えてきました。近年、広島県では「産業国際化型特別地域」制度を利用し、広島製造産業省立大学を設置し、国際コミュニティとの連携による教育の実現を目指しています。また、東京都では、モバイリティやマニュアファクチ

,id,instruction,input,output
0,1,貴方はアシスタントです．与えられた文章を適切に要約してください．,日本の情報通信技術（ICT）が国内経��PERTY及び就IT業界への意識的参入やその妨げに関...,調査結果より日本の情報通信技術（ICT）が国内経済及び就IT業界への意識的参入に関する問題点...
1,2,貴方はアシスタントです．与えられた文章を適切に要約してください．,日本の経済成長は明確な安定性を示しており、世界的なサプライチェーン危機やCOVID-19パン...,日本の経済成長は安定しており、COVID-19パンデミックにも耐えてきています。近年、日本で...
2,3,貴方はアシスタントです．与えられた文章を適切に要約してください．,環境、社会、治理（ESG）投資は、リスク回避と長期的投資利益を得る上で重要な一つであり、世界...,ESG投資は、リスク回避と長期的投資利益を得る上で重要で、世界的に注目を集めています。日本企...
0,1,貴方はアシスタントです．与えられた文章を適切に要約してください．,私たちの企業は、顧客満足度を向上させるためにクライアントサポートチームを強化しています。新し...,私たちの企業は、顧客満足度の向上を目指してクライアントサポートチームを強化しています。新規採...
1,2,貴方はアシスタントです．与えられた文章を適切に要約してください．,最近の研究では、働き方の多様性が企業の成果に大きな影響を与えることが示されました。特に、リモ...,働き方の多様性は企業の成果に大きな影響を与えています。リモートワークやフレックスタイムは生産...


##1-3.データの保存

In [ ]:
import datetime

# Get the current datetime
now = datetime.datetime.now()

# Format the datetime as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with the timestamp
filename = f"/generated_data_{timestamp}.xlsx"

# Save the DataFrame to an Excel file with the timestamped filename
df_final.to_excel(G_FOLDER + filename, index=False)

print(f"File saved as: {filename}")

File saved as: /generated_data_2024-05-18_14-35-16.xlsx
